# Libraries

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# import the required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, precision_recall_curve, auc
from sklearn.feature_selection import f_classif
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from scipy.stats import chi2_contingency
import scorecardpy as sc
from scorecardpy.LogisticRegStats import LogisticRegStats
import random as rd
import re
from IPython.display import display
from matplotlib.backends.backend_pdf import PdfPages

# Data

In [ ]:
# data prepare ------
# load germancredit data
smp_full = sc.germancredit()
smp_full['will_default'] = smp_full['creditability'].apply(lambda x: 1 if x == 'bad' else 0)
smp_full = smp_full.loc[:,smp_full.columns != 'creditability']
smp_full.loc[0:99, 'credit.amount'] = np.nan
smp_full.loc[0:99, 'purpose'] = np.nan
smp_full.loc[100:109, 'will_default'] = np.nan

for i in range(5):
    smp_full = pd.concat([smp_full, smp_full])
smp_full['RepDate_End'] = np.random.randint(1, 73, smp_full.shape[0])
smp_full = smp_full.reset_index(drop=True)

In [ ]:
smp_full = smp_full.rename(columns={"will_default": "target"})
smp_full

# 1. Preliminary analysis of variables (missings, outliers, concentration/distribution) - based on smp_full

In [ ]:
# columns that are not variables
var_skip = ['target','RepDate_End']
# special values for numeric variables - TBD
spl_val = []
# list of variables by type (numeric variables with less than 10 unique values are considered as categorical)
var_cat, var_num = sc.var_types(smp_full, var_skip)

In [ ]:
# heatmap for the missing values
percent_missing = smp_full.loc[:, var_cat+var_num].isna().sum() * 100 / len(smp_full)
percent_missing = pd.DataFrame({'column':percent_missing.index, 'percent_missing':percent_missing.values})
percent_missing.sort_values('percent_missing', ascending=False, inplace=True)
percent_missing.reset_index(drop=True)

plt.figure(figsize=(10,6))
sns.heatmap(smp_full[percent_missing.column].isna().transpose(),
            cmap="YlGnBu",
            cbar_kws={'label': 'Missing Data'})
plt.savefig("1_1_missings_heatmap.png", dpi=100, bbox_inches = "tight")

In [ ]:
#round missings
#thresholds as params
# warning checks
var_cat_summary, var_num_summary = sc.var_pre_analysis(smp_full, var_cat, var_num, spl_val, hhi_low=0.05, hhi_high=0.95, min_share=0.05)

writer = pd.ExcelWriter('1_2_preliminary_analysis.xlsx', engine='xlsxwriter')
var_cat_summary.to_excel(writer, sheet_name='var_cat_summary')
var_num_summary.to_excel(writer, sheet_name='var_num_summary')
writer.save()

display(var_cat_summary)
display(var_num_summary)

In [ ]:
#treatment of nan - median for numeric and 'Missing' for string
for var, dt in smp_full.dtypes.items():
    if var not in var_skip and smp_full[var].isna().sum() > 0:
        print(var,smp_full[var].isna().sum()) 
        if dt.name == 'category':
            smp_full[var] = smp_full[var].cat.add_categories('Missing').fillna('Missing')
            print('Missing')
        if dt.name == 'object':
            smp_full[var] = smp_full[var].fillna('Missing')
            print('Missing')
        else: 
            print(smp_full[var].median())
            smp_full[var] = smp_full[var].fillna(smp_full[var].median())

In [ ]:
# distribution for categorical variables with extract to pdf
sc.var_cat_distr(smp_full, var_cat, '1_3_categorical_vars_distribution.pdf', groupby='target')

In [ ]:
sc.var_num_distr(smp_full, var_num, '1_4_numerical_vars_distribution.pdf', groupby='target')

# 2. Development sample creation

In [ ]:
# selection of the development window 
sorted_date = sorted(smp_full['RepDate_End'].unique())
del sorted_date[-12:]
smp_dev = smp_full.loc[smp_full['RepDate_End'].isin(sorted_date)]
# smp_dev = smp_full.loc[smp_full['RepDate_End'] < 20210700]
# smp_dev = smp_full.loc[smp_full['RepDate_End'].isin(sorted_date)]

In [ ]:
# check target
print(smp_dev.groupby('target', dropna=False).size())

In [ ]:
# delete records with blank target
smp_dev = smp_dev[smp_dev['target'].notna()]

In [ ]:
# selection of variables that will be used for the development
smp_dev = smp_dev[var_cat+var_num+['target']+['RepDate_End']]

#smp_dev = smp_full.loc[smp_dev['prod_grp'] == 'Mortgage']

# train/test split as 80/20
train, test = sc.split_df(smp_dev, ratio=0.8, seed=123).values()
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [ ]:
# train/test sample size !update
pd.DataFrame({'train':pd.Series(train.groupby('target', dropna=False).size()),
              'test':pd.Series(test.groupby('target', dropna=False).size())})

# 3. Automated binning

In [ ]:
# binning
fine_class, coarse_class = sc.woebin(train, y = 'target', x = var_cat + var_num, init_count_distr = 0.05)

In [ ]:
# extracting binning results to excel !update
pd.concat(fine_class.values()).reset_index(drop=True).to_excel('3_1_fine_classing.xlsx')
pd.concat(coarse_class.values()).reset_index(drop=True).to_excel('3_2_coarse_classing_auto.xlsx')

In [ ]:
# iv for variables after automated binning !update
coarse_class_iv = sc.vars_iv(var_cat + var_num, coarse_class)
coarse_class_iv.to_excel('3_3_coarse_classing_auto_iv.xlsx')
coarse_class_iv

In [ ]:
# binning visualization
# sc.woebin_plot(coarse_class)

# 4. Binning adjustments 

In [ ]:
# manual review and adjustment of binning (first line should uncommented if manual adjustments are needed to be done) !update
# breaks_list = sc.woebin_adj(train, y="target", bins=coarse_class, fine_bins=fine_class, adj_all_var=False, save_breaks_list='3_4_breaks_list_adj.py')
%run 3_4_breaks_list_adj.py

In [ ]:
# variables excluded based on binning results
vars_bin_excl = []

In [ ]:
# update of coarse classing table (fine classing is relevant only for automated binning)
fine_class_adj, coarse_class_adj = sc.woebin(train, y = 'target', x = var_cat + var_num, breaks_list = breaks_list, init_count_distr = 0.05)

In [ ]:
# applying woe transformations on train and test samples 
train_woe = sc.woebin_ply(train, bins=coarse_class_adj)
test_woe = sc.woebin_ply(test, bins=coarse_class_adj)

# defining woe variables
vars_woe = []
for i in var_cat + var_num:
    if i not in vars_bin_excl:
        vars_woe.append(i+'_woe')

In [ ]:
# results of the final coarse classing after manual adjustments !update
pd.concat(coarse_class_adj.values()).reset_index(drop=True).to_excel('3_5_coarse_classing_final.xlsx')
coarse_class_adj_iv = sc.vars_iv(var_cat + var_num, coarse_class_adj)
coarse_class_adj_iv.to_excel('3_6_coarse_classing_final_iv.xlsx')
coarse_class_adj_iv

In [ ]:
# IV for variables by defined subsamples (period, product etc.)
sc.iv_group(train_woe, vars_woe, groupby='RepDate_End')

# 5. Correlation analysis

In [ ]:
# correlation matrix
train_woe_corr = train_woe[vars_woe].corr()
train_woe_corr.to_excel('5_1_correlation_matrix.xlsx')
train_woe_corr

In [ ]:
# plotting correlation heatmap
plt.figure(figsize=(20,12))
sns.heatmap(train_woe[vars_woe].corr(), cmap="YlGnBu", annot=True)
  
# displaying heatmap
plt.savefig('5_2_correlation_heatmap.png')
plt.show()

In [ ]:
# exclusions by corr > 0.7
vars_corr_excl = []

# 6. Logistic regression

In [ ]:
# values in the test sample that are not present in train
#print(np.am=ny(no.isnan(test_woe)))
test_woe = test_woe.dropna()

In [ ]:
vars_woe = []
for i in var_cat + var_num:
    if i not in vars_bin_excl + vars_corr_excl:
        vars_woe.append(i+'_woe')

In [ ]:
# target and variables
y_train = train_woe['target']
X_train = train_woe[vars_woe]
y_test = test_woe['target']
X_test = test_woe[vars_woe]

In [ ]:
# logistic regression ------
lr = LogisticRegression(penalty='l1', C=0.9, solver='saga', n_jobs=-1)
lr.fit(X_train, y_train)
# lr.coef_
# lr.intercept_

In [ ]:
# predicted proability
train_pred = lr.predict_proba(X_train)[:,1]
test_pred = lr.predict_proba(X_test)[:,1]
# performance ks & roc ------
train_perf = sc.perf_eva(y_train, train_pred, title = "train")
test_perf = sc.perf_eva(y_test, test_pred, title = "test")

In [ ]:
# train bad rate
train_br = {}
train_br['Total'] = y_train.count()
train_br['Bads'] = int(y_train.sum())
train_br['Bad Rate'] = round(train_br['Bads']/train_br['Total'], 4)
# test bad rate
test_br = {}
test_br['Total'] = y_test.count()
test_br['Bads'] = int(y_test.sum())
test_br['Bad Rate'] = round(test_br['Bads']/test_br['Total'], 4)
test_br
# combining bad rate with performance
perf = pd.DataFrame({'train':pd.Series(dict(list(train_br.items()) + list(train_perf.items()))),
                         'test':pd.Series(dict(list(test_br.items()) + list(test_perf.items())))})
perf = perf.loc[~perf.index.isin(['pic'])]
perf.to_excel('6_1_perf_train_test.xlsx')
perf

In [ ]:
# logistic regression with stats
lr2 = LogisticRegStats(penalty='l1', C=0.9, solver='saga', n_jobs=-1)
lr2.fit(X_train, y_train)

# calculating p-values and exporting to excel
lr_output = pd.DataFrame({
                'Variable': ['intercept'] + X_train.columns.tolist(),
                'Coefficient': [lr2.model.intercept_[0]] + lr2.model.coef_[0].tolist(),
                'P-value': [0] + lr2.p_values
                })

lr_output.to_excel('6_2_regr_output.xlsx')
lr_output

In [ ]:
# score ------
card = sc.scorecard(coarse_class_adj, lr, X_train.columns, start_zero=True)
# credit score
train_score  = sc.scorecard_ply(train, card, print_step=0)
test_score = sc.scorecard_ply(test, card, print_step=0)
# output to excel
scorecard_points = pd.concat(card, ignore_index=True)
scorecard_points.to_excel("6_3_scorecard_points.xlsx", sheet_name='scorecard_points')

# 7. Testing

In [ ]:
# samples scoring
smp_dev_score = sc.woebin_ply(smp_dev, bins=coarse_class_adj)
smp_dev_score['score'] = sc.scorecard_ply(smp_dev, card, print_step=0)

smp_full_score = sc.woebin_ply(smp_full, bins=coarse_class_adj)
smp_full_score['score'] = sc.scorecard_ply(smp_full, card, print_step=0)

# adding target
train_score['target'] = train['target']
test_score['target'] = test['target']

In [ ]:
# Bad Rate over time
gini_ot = smp_dev_score[['RepDate_End', 'target']].groupby(['RepDate_End']).agg(['count', 'sum' ])
gini_ot = gini_ot.rename(columns={"count": "Total", "sum": "Bads"})
gini_ot.columns = gini_ot.columns.droplevel(0)
gini_ot['Bad Rate'] = (gini_ot['Bads'] / gini_ot['Total'])
# adding Gini over time
gini_ot['Gini'] = sc.gini_over_time(smp_dev_score, 'target', ['score'], 'RepDate_End')

# Gini for vars train/test
gini_vars_train = sc.gini_vars(train_woe, 'target', vars_woe, 'Train')
gini_vars_test = sc.gini_vars(test_woe, 'target', vars_woe, 'Test')
gini_vars_train_test = pd.merge(gini_vars_train, gini_vars_test, on = "Variable")

# Gini for vars over time
gini_vars_ot = sc.gini_over_time(smp_dev_score, 'target', vars_woe, 'RepDate_End')
                                 
# defining score ranges on train sample
_, brk = pd.cut(train_score['score'], bins=10, retbins=True, duplicates='drop')
brk = brk.round(decimals=2)
brk = list(filter(lambda x: x>np.nanmin(train_score['score']) and x<np.nanmax(train_score['score']), brk))
brk = [np.nanmin(smp_full_score['score'])] + sorted(brk) + [np.nanmax(smp_full_score['score'])]
# applying score ranges on train, test adn full samples
train_score['score_range'] = pd.cut(train_score['score'], bins=brk, include_lowest=False)
test_score['score_range'] = pd.cut(test_score['score'], bins=brk, include_lowest=False)
smp_full_score['score_range'] = pd.cut(smp_full_score['score'], bins=brk, include_lowest=False)
# score distribution for train/test
train_distr = sc.score_distr(train_score, 'target', 'score', 'score_range')
test_distr = sc.score_distr(test_score, 'target', 'score', 'score_range')
                                 
# PSI over time (score_range) 
psi_ot = sc.psi_over_time(train_score, smp_full_score, ['score_range'], 'RepDate_End')
                                 
# PSI for WoE variables over time
psi_vars_ot = sc.psi_over_time(train_woe, smp_full_score, vars_woe, 'RepDate_End')
                                 
# calculating hhi for train/test
train_hhi = sc.hhi(train_score['score_range'].astype(str))
test_hhi = sc.hhi(test_score['score_range'].astype(str))
hhi_train_test = pd.DataFrame({
        'train': [train_hhi], 
        'test': [test_hhi]
    }, index = ['hhi'])
                                 
# calculating hhi over time
hhi_ot = smp_full_score.groupby('RepDate_End').agg({'score_range': sc.hhi})
hhi_ot = hhi_ot.rename(columns={"score_range": "HHI"})

In [ ]:
# assigning ratings
bins = [0,500,540,580,620,660,700,740,780,1000]
labels = ['4.5','4.0','3.5','3.0','2.5','2.0','1.5','1.0','0.5']
smp_dev_score['rating'] = pd.cut(smp_dev_score['score'], bins=bins, labels=labels, include_lowest=True)
smp_full_score['rating'] = pd.cut(smp_full_score['score'], bins=bins, labels=labels, include_lowest=True)

# DR by rating on dev sample
DR_rating = smp_dev_score[['rating', 'target']].groupby(['rating']).agg(['count', 'sum' ])
DR_rating = DR_rating.rename(columns={"count": "Total", "sum": "Bads"})
DR_rating.columns = DR_rating.columns.droplevel(0)
DR_rating['DR'] = (DR_rating['Bads'] / DR_rating['Total'])

# DR by rating over time on full sample
DR_rating_ot = smp_full_score[['RepDate_End', 'rating', 'target']].groupby(['RepDate_End', 'rating']).agg(['count', 'sum' ])
DR_rating_ot = DR_rating_ot.rename(columns={"count": "Total", "sum": "Bads"})
DR_rating_ot.columns = DR_rating_ot.columns.droplevel(0)
DR_rating_ot = DR_rating_ot.reset_index()
DR_rating_ot['DR'] = (DR_rating_ot['Bads'] / DR_rating_ot['Total'])
# set DR to null for last 12 months
sorted_date = sorted(DR_rating_ot['RepDate_End'].unique())
DR_rating_ot['DR'].loc[DR_rating_ot['RepDate_End'].isin(sorted_date[-12:])] = np.nan


In [ ]:
# exporting results to excel
writer = pd.ExcelWriter('7_1_testing_results.xlsx', engine='xlsxwriter')
gini_ot.to_excel(writer, sheet_name='Gini_OT')
gini_vars_train_test.to_excel(writer, sheet_name='Gini_Vars')
gini_vars_ot.to_excel(writer, sheet_name='Gini_Vars_OT')
train_distr.to_excel(writer, sheet_name='Distr_Train')
test_distr.to_excel(writer, sheet_name='Distr_Test')
psi_ot.to_excel(writer, sheet_name='PSI_OT')
psi_vars_ot.to_excel(writer, sheet_name='PSI_Vars_OT')
hhi_train_test.to_excel(writer, sheet_name='HHI')
hhi_ot.to_excel(writer, sheet_name='HHI_OT')
DR_rating.to_excel(writer, sheet_name='DR_rating')
DR_rating_ot.to_excel(writer, sheet_name='DR_rating_ot')
writer.save()

In [ ]:
# DR vs PD by product over time
def pd_from_score(score, points0=540, odds0=1/9, pdo=40):
    b = pdo/np.log(2)
    a = points0 + b*np.log(odds0)
    pd = 1/(1+np.exp(score/b - a/b))
    return pd

smp_dev_score['pd'] = pd_from_score(smp_dev_score['score'])

dr_ot = smp_dev_score[['RepDate_End', 'rating', 'target']].groupby(['RepDate_End', 'rating']).agg(['count', 'sum' ])
dr_ot = dr_ot.rename(columns={"count": "Total", "sum": "Bads"})
dr_ot.columns = dr_ot.columns.droplevel(0)
dr_ot['Bad Rate'] = (dr_ot['Bads'] / dr_ot['Total'])

dr_ot2 = smp_dev_score[['RepDate_End', 'rating', 'pd']].groupby(['RepDate_End', 'rating']).agg(['count', 'sum' ])
dr_ot2 = dr_ot2.rename(columns={"count": "Total", "sum": "Bads"})
dr_ot2.columns = dr_ot2.columns.droplevel(0)
dr_ot2['PD'] = (dr_ot2['Bads'] / dr_ot2['Total'])
dr_ot['PD'] = dr_ot2['PD']

dr_ot = dr_ot.swaplevel()

dr_ot.to_excel('7_2_DR_ot_products.xlsx')

# 8. Recalibration

In [ ]:
# preparing sample for recalibration
train_score  = sc.scorecard_ply(train, card, print_step=0)
train_score['target'] = train['target']
train_score['pd_regr'] = sc.pd_from_score(train_score['score'])

test_score  = sc.scorecard_ply(test, card, print_step=0)
test_score['target'] = test['target']
test_score['pd_regr'] = sc.pd_from_score(test_score['score'])

smp_calib_score = train_score.append(test_score)

# assigning ratings
bins = [0,500,540,580,620,660,700,740,780,1000]
labels = ['4.5','4.0','3.5','3.0','2.5','2.0','1.5','1.0','0.5']
smp_calib_score['rating'] = pd.cut(smp_calib_score['score'], bins=bins, labels=labels, include_lowest=True)

In [ ]:
def calibration(smp, score='score', target='target', points0=540, odds0=1/9, pdo=40):
    b = pdo/np.log(2) 
    a = points0 + b*np.log(odds0)
    log_odds = a/b - smp[score]/b
    x = log_odds.to_numpy().reshape(-1, 1)
    y = smp[target].to_numpy()
    
    lr_calib = LogisticRegression(penalty='none', solver='newton-cg', n_jobs=-1)
    lr_calib.fit(x, y)

    pd_calib = lr_calib.predict_proba(x_calib)[:,1]
    log_odds_calib = np.log((1-pd_calib)/(pd_calib))

    intercept_calib = points0 + (np.log(odds0) - lr_calib.intercept_) * pdo / np.log(2)
    slope_calib = lr_calib.coef_ * pdo / np.log(2)
    intercept_calib = intercept_calib[0]
    slope_calib = slope_calib[0,0]

    intercept = intercept_calib - slope_calib*a/b
    slope = slope_calib/b
    return intercept, slope

In [ ]:
intercept, slope = calibration(smp_calib_score, score='score', target='target')
print(intercept, slope)

In [ ]:
smp_calib_score['score_new'] = smp_calib_score['score']*slope + intercept
smp_calib_score['score_new'] = smp_calib_score['score_new'].astype(int)
smp_calib_score['rating_new'] = pd.cut(smp_calib_score['score_new'], bins=bins, labels=labels, include_lowest=True)